In [31]:
import numpy as np
import pandas as pd
import plotly.express as px
from operator import methodcaller
import os
from sklearn import linear_model
from util import *
import math

In [32]:

files = [f for f in os.listdir('.') if os.path.isfile(f)]
data_files = []

for f in files:
    if "callibrate" in f:
        callibration_file = f
    elif "csv" in f:
        data_files.append(f)

In [33]:
vis_angles_callib = vis_angles_from_file(callibration_file)
X1 = vis_angles_callib[['gaze_vis_x','gaze_vis_y']]
y1 = vis_angles_callib['target_vis_x']

X2 = vis_angles_callib[['gaze_vis_x','gaze_vis_y']]
y2 = vis_angles_callib['target_vis_y']

In [34]:
x_regr = linear_model.LinearRegression()
x_regr.fit(X1.values, y1.values)
coeffs1 = x_regr.coef_

y_regr = linear_model.LinearRegression()
y_regr.fit(X2.values, y2.values)
coeffs2 = y_regr.coef_

In [41]:
if not os.path.exists('recal'):
    os.makedirs('recal')

for f in data_files:
    df = pd.read_csv(f)
    indexExpStarts = 0
    for index, row in df.iterrows():
        if row['PathIDX'] != 99:
            indexExpStarts = index
            break

    indexExpStarts = max(0, indexExpStarts-500)
    df = df.drop(range(indexExpStarts))
    df = df[df["left_right_eye_is_blinking"].str.contains("True") == False]
    df['seconds'] = df['seconds'].apply(lambda x: x-df['seconds'].iat[0])
    df['frame'] = df['frame'].apply(lambda x: x-df['frame'].iat[0])

    df.index = np.arange(0, len(df))
    df['gaze_x_recal'] = df.apply(lambda row: x_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)
    df['gaze_y_recal'] = df.apply(lambda row: y_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)

    df = df[['frame','PathIDX','seconds','gaze_vis_x','gaze_vis_y','gaze_x_recal','gaze_y_recal', 'target_vis_x', 'target_vis_y']]
    df.to_csv("recal/"+f[:-4]+'_recal.csv')

    spatial_euc_errors(df)